In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install KoNLPy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 13.9 MB/s eta 0:00:00


In [2]:
import numpy as np

In [3]:
import pandas as pd

stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

### OTA 데이터 불러오기

In [4]:
import pandas as pd

df_agoda = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_agoda', encoding='utf-8')
df_airbnb = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_airbnb', encoding='utf-8')
df_booking = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_booking', encoding='utf-8')
df_dailyhotel = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_dailyhotel', encoding='utf-8')
df_hotelsdotcom = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_hotelsdotcom', encoding='utf-8')
df_myrealtrip = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_myrealtrip', encoding='utf-8')
df_yanol = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_yanol', encoding='utf-8')
df_yeogi = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_yeogi', encoding='utf-8')

In [5]:
all_OTA = pd.concat([df_agoda,df_airbnb,df_booking,df_dailyhotel,df_hotelsdotcom,df_myrealtrip,df_yanol,df_yeogi])

all_OTA.shape

(25127, 5)

In [6]:
all_OTA

,date,content,score,at,sentiment
0,0,사용 후기를 통해 쉽게 예매할 수 있어 좋아요,5,20221231223041,1
1,1,세금 포함 최종 가격 올려주는 기능은 없나요 혼동되네요,4,20221231142125,1
2,2,저렴한 가격 최고의 어플,5,20221231141747,1
3,3,저렴한 가격에 좋은 곳을 소개받을 수 있어 좋아요,5,20221231093122,1
4,4,내용들이 선택하기 좋게 잘 되어 있어요,5,20221231030646,1
...,...,...,...,...,...
3496,3496,너무 조아여 가격 저렴해요,5,20200104093150,1
3497,3497,환불 자체를 아예 안 해주는 편이네 요항상 방 배정이랑 방일 적실 수 환불 전혀 안...,1,20200103215249,0
3498,3498,호텔 예약했는데 알고 보니 입실시간이 저녁 시네요 황당한 네 그기에 당일 예약이라 ...,1,20200103160905,0
3499,3499,다운로드 대기 중이라고만 뜨고 계속 다운이 안 되는데 해결할 방법이 있나요,1,20200102095048,0


In [7]:
count_zero = len(all_OTA[all_OTA['sentiment'] == 0])
count_one = len(all_OTA[all_OTA['sentiment'] == 1])
print("Sentiment 값이 0인 데이터 개수:", count_zero)
print("Sentiment 값이 1인 데이터 개수:", count_one)

Sentiment 값이 0인 데이터 개수: 6202
Sentiment 값이 1인 데이터 개수: 18925


### 데이터 정제 함수 생성

In [ ]:
def preprocess_df(df):
    df = df.drop(columns=['date', 'score', 'sentiment'])
    print(df.isnull().values.any())
    print(df.isnull().sum())
    df = df.dropna(how='any')
    print(df.isnull().values.any())
    df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    df['content'] = df['content'].str.replace('^ +', "")
    df['content'].replace('', np.nan, inplace=True)
    df_2020 = df[(df['at'] >= 20200000000000) & (df['at'] < 20210000000000)].reset_index(drop=True)
    df_2021 = df[(df['at'] >= 20210000000000) & (df['at'] < 20220000000000)].reset_index(drop=True)
    df_2022 = df[(df['at'] >= 20220000000000) & (df['at'] < 20230000000000)].reset_index(drop=True)
    df_2020 = df_2020.astype(str)
    df_2021 = df_2021.astype(str)
    df_2022 = df_2022.astype(str)
    display(df_2020.shape)
    display(df_2021.shape)
    display(df_2022.shape)
    return [df_2020, df_2021, df_2022]

# dataframes = [df_agoda, df_airbnb, df_booking, df_dailyhotel, df_hotelsdotcom, df_myrealtrip, df_yanol, df_yeogi]
# OTA_names = ['df_agoda', 'df_airbnb', 'df_booking', 'df_dailyhotel', 'df_hotelsdotcom', 'df_myrealtrip', 'df_yanol', 'df_yeogi']

# for df, OTA_name in zip(dataframes, OTA_names):
#     df_2020, df_2021, df_2022 = preprocess_df(df)
#     # 이후 각 데이터프레임(df_2020, df_2021, df_2022)을 활용하여 분석 수행

### SVM 불러오기

In [ ]:
from joblib import load

# 모델이 저장된 경로와 파일명을 지정합니다.
model_path = "/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/감성분석모델/svm_model.pkl"

# 모델을 불러옵니다.
svm_model = load(model_path)

In [ ]:
import pickle

with open('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/감성분석모델/vocabulary.pkl', 'rb') as f:
    vocabulary = pickle.load(f)


### 각 TF-IDF

In [ ]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

def predict_sentiment(texts, model, vocabulary):
    okt = Okt()
    
    def preprocess(text):
        words = okt.morphs(text, stem=True)
        words = [word for word in words if word not in stopwords]
        return ' '.join(words)
    
    def calculate_tfidf(text, vocabulary):
        tfidf = TfidfVectorizer(vocabulary=vocabulary)
        tfidf.fit(text)
        return tfidf.transform(text)
    
    def predict_sentiment_for_text(text, model, vocabulary):
        preprocessed_text = preprocess(text)
        tfidf = calculate_tfidf([preprocessed_text], vocabulary)
        score = model.predict(tfidf)[0]
        return (score * 100) - 50
    
    scores = [predict_sentiment_for_text(text, model, vocabulary) for text in texts]
    return sum(scores) / len(scores) # 입력 text의 점수 평균값 리턴

## agoda

In [ ]:
df_agoda_2020, df_agoda_2021, df_agoda_2022 = preprocess_df(df_agoda)

True
content    4
at         0
dtype: int64
False


<ipython-input-8-cc8037155fa3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-8-cc8037155fa3>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('^ +', "")


(682, 2)

(900, 2)

(1096, 2)

In [ ]:
OTA_name = [df_agoda_2020, df_agoda_2021, df_agoda_2022]
for OTA_df in OTA_name:
    avg_score = predict_sentiment(OTA_df['content'], svm_model, vocabulary)
    print(f"평균 점수: {avg_score:.2f}")

평균 점수: 25.07
평균 점수: 34.67
평균 점수: 19.71


### airbnb

In [ ]:
df_airbnb_2020, df_airbnb_2021, df_airbnb_2022 = preprocess_df(df_airbnb)

True
content    2
at         0
dtype: int64
False


<ipython-input-8-cc8037155fa3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-8-cc8037155fa3>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('^ +', "")


(796, 2)

(606, 2)

(604, 2)

In [ ]:
OTA_name = [df_airbnb_2020, df_airbnb_2021, df_airbnb_2022]
for OTA_df in OTA_name:
    avg_score = predict_sentiment(OTA_df['content'], svm_model, vocabulary)
    print(f"평균 점수: {avg_score:.2f}")

평균 점수: 21.73
평균 점수: 3.47
평균 점수: 3.81


### bookings

In [ ]:
df_booking_2020, df_booking_2021, df_booking_2022 = preprocess_df(df_booking)

False
content    0
at         0
dtype: int64
False


<ipython-input-8-cc8037155fa3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-8-cc8037155fa3>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('^ +', "")


(616, 2)

(267, 2)

(389, 2)

In [ ]:
OTA_name = [df_booking_2020, df_booking_2021, df_booking_2022]
for OTA_df in OTA_name:
    avg_score = predict_sentiment(OTA_df['content'], svm_model, vocabulary)
    print(f"평균 점수: {avg_score:.2f}")

평균 점수: 42.53
평균 점수: 39.51
평균 점수: 33.29


### 데일리호텔

In [ ]:
df_dailyhotel_2020, df_dailyhotel_2021, df_dailyhotel_2022 = preprocess_df(df_dailyhotel)

True
content    8
at         0
dtype: int64
False


<ipython-input-8-cc8037155fa3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-8-cc8037155fa3>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('^ +', "")


(4412, 2)

(1775, 2)

(89, 2)

In [ ]:
OTA_name = [df_dailyhotel_2020, df_dailyhotel_2021, df_dailyhotel_2022]
for OTA_df in OTA_name:
    avg_score = predict_sentiment(OTA_df['content'], svm_model, vocabulary)
    print(f"평균 점수: {avg_score:.2f}")

평균 점수: 47.08
평균 점수: 46.23
평균 점수: 10.67


### 호텔스닷컴

In [ ]:
df_hotelsdotcom_2020, df_hotelsdotcom_2021, df_hotelsdotcom_2022 = preprocess_df(df_hotelsdotcom)

False
content    0
at         0
dtype: int64
False


<ipython-input-8-cc8037155fa3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-8-cc8037155fa3>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('^ +', "")


(519, 2)

(341, 2)

(706, 2)

In [ ]:
OTA_name = [df_hotelsdotcom_2020, df_hotelsdotcom_2021, df_hotelsdotcom_2022]
for OTA_df in OTA_name:
    avg_score = predict_sentiment(OTA_df['content'], svm_model, vocabulary)
    print(f"평균 점수: {avg_score:.2f}")

평균 점수: 33.62
평균 점수: 29.47
평균 점수: -1.70


### 마이리얼트립

In [ ]:
df_myrealtrip_2020, df_myrealtrip_2021, df_myrealtrip_2022 = preprocess_df(df_myrealtrip)

True
content    1
at         0
dtype: int64
False


<ipython-input-8-cc8037155fa3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-8-cc8037155fa3>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('^ +', "")


(401, 2)

(159, 2)

(139, 2)

In [ ]:
OTA_name = [df_myrealtrip_2020, df_myrealtrip_2021, df_myrealtrip_2022]
for OTA_df in OTA_name:
    avg_score = predict_sentiment(OTA_df['content'], svm_model, vocabulary)
    print(f"평균 점수: {avg_score:.2f}")

평균 점수: 38.28
평균 점수: 16.04
평균 점수: -5.40


### 야놀자

In [ ]:
df_yanol_2020, df_yanol_2021, df_yanol_2022 = preprocess_df(df_yanol)

True
content    13
at          0
dtype: int64
False


<ipython-input-8-cc8037155fa3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-8-cc8037155fa3>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('^ +', "")


(2689, 2)

(1948, 2)

(2464, 2)

In [ ]:
OTA_name = [df_yanol_2020, df_yanol_2021, df_yanol_2022]
for OTA_df in OTA_name:
    avg_score = predict_sentiment(OTA_df['content'], svm_model, vocabulary)
    print(f"평균 점수: {avg_score:.2f}")

평균 점수: 35.09
평균 점수: 20.89
평균 점수: 33.00


### 여기어때

In [ ]:
df_yeogi_2020, df_yeogi_2021, df_yeogi_2022 = preprocess_df(df_yeogi)

True
content    12
at          0
dtype: int64
False


<ipython-input-8-cc8037155fa3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-8-cc8037155fa3>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('^ +', "")


(1025, 2)

(902, 2)

(1562, 2)

In [ ]:
OTA_name = [df_yeogi_2020, df_yeogi_2021, df_yeogi_2022]
for OTA_df in OTA_name:
    avg_score = predict_sentiment(OTA_df['content'], svm_model, vocabulary)
    print(f"평균 점수: {avg_score:.2f}")

평균 점수: -10.00
평균 점수: -12.08
평균 점수: 12.80
